In [32]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt #to plot charts
import seaborn as sns #used for data visualization
import warnings #avoid warning flash
warnings.filterwarnings('ignore')


import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

### 1. ¿Qué es la Diabetes?
La diabetes es una enfermedad crónica que ocurre cuando el páncreas deja de producir insulina o cuando el cuerpo no puede utilizar adecuadamente la insulina que genera. La insulina es una hormona esencial que permite que la glucosa en la sangre entre en las células para ser utilizada como energía. Si no se controla, la diabetes puede provocar complicaciones graves, como enfermedades cardíacas, daño renal, problemas de visión y neuropatías.

El uso de técnicas de `Machine Learning` nos permite analizar patrones en los datos médicos para predecir la probabilidad de que una persona tenga diabetes. En este proyecto, aplicaremos modelos de aprendizaje automático para clasificar a los pacientes en dos categorías: aquellos con diabetes y aquellos sin la enfermedad.

El conjunto de datos utilizado contiene las siguientes variables independientes (características médicas) y una variable dependiente (resultado):

**Variables independientes:**

- `Pregnancies:` Número de veces que la mujer ha estado embarazada.
- `Glucose:` Concentración de glucosa en plasma a las 2 horas en una prueba de tolerancia a la glucosa oral.
- `BloodPressure:` Presión arterial diastólica (mm Hg).
- `SkinThickness:` Grosor del pliegue cutáneo del tríceps (mm).
- `Insulin:` Nivel de insulina en suero a las 2 horas (mu U/ml).
- `BMI:` Índice de Masa Corporal (peso en kg/altura en m²).
- `DiabetesPedigreeFunction:` Puntaje que indica la probabilidad de diabetes basada en antecedentes familiares.
- `Age:` Edad (años).

**Variable dependiente:**
- `Outcome:` Indica si la persona tiene diabetes (1) o no (0).

In [2]:
df=pd.read_csv("diabetes.csv")

In [3]:
df.shape

(768, 9)

In [4]:
df.head(4)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [6]:
df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


### 2. Data Cleaning
La limpieza de datos es un paso fundamental en cualquier análisis o modelo de Machine Learning, ya que garantiza la calidad y precisión de los resultados. Los datos suelen contener valores nulos, duplicados, errores tipográficos o inconsistencias que pueden afectar la interpretación y el rendimiento de los modelos.

In [ ]:
df=df.drop_duplicates()

In [ ]:
df.isnull().sum()

In [23]:
str(len(df[df['BloodPressure']==0]))

'35'

In [27]:
print(f'Cantidad de registros cero de BloodPressure: ', df[df['BloodPressure']==0].shape[0])
print(f'Cantidad de registros cero de Glucose:', df[df['Glucose']==0].shape[0])
print(f'Cantidad de registros cero de SkinThickness:', df[df['SkinThickness']==0].shape[0])
print(f'Cantidad de registros cero de Insulin:',df[df['Insulin']==0].shape[0])
print(f'Cantidad de registros cero de BMI:', df[df['BMI']==0].shape[0])

Cantidad de registros cero de BloodPressure:  35
Cantidad de registros cero de Glucose: 5
Cantidad de registros cero de SkinThickness: 227
Cantidad de registros cero de Insulin: 374
Cantidad de registros cero de BMI: 11


Debido a que hay una variables en mi data set con valores ceros que son poco razonables se hara un remplazo de estos

Algunas de las columnas tienen una distribución sesgada, por lo que la media se ve más afectada por los valores atípicos que la mediana. La glucosa y la presión arterial tienen distribuciones normales, por lo que reemplazamos los valores 0 en esas columnas por el valor medio. El grosor de la piel, la insulina y el IMC tienen distribuciones sesgadas, por lo que la mediana es una mejor opción, ya que se ve menos afectada por los valores atípicos.
Ayudame a redactar mejor esta idea

In [28]:
df['Glucose']=df['Glucose'].replace(0,df['Glucose'].mean())
df['BloodPressure']=df['BloodPressure'].replace(0,df['BloodPressure'].mean())
df['SkinThickness']=df['SkinThickness'].replace(0,df['SkinThickness'].median())
df['Insulin']=df['Insulin'].replace(0,df['Insulin'].median())
df['BMI']=df['BMI'].replace(0,df['BMI'].median())

#### Distribucion de las variables

In [45]:

fig = make_subplots(rows=3, cols=3, subplot_titles=('Scatter Plot', 'Histogram'))

fig1 = px.histogram(x=df['Pregnancies'])
fig2 = px.histogram(x=df['Glucose'])
fig3 = px.histogram(x=df['BloodPressure'])
fig4 = px.histogram(x=df['Insulin'])
fig5 = px.histogram(x=df['BMI'])
fig6 = px.histogram(x=df['DiabetesPedigreeFunction'])
fig7 = px.histogram(x=df['Age'])
fig8 = px.histogram(x=df['SkinThickness'])


for trace in fig1.data:
    fig.add_trace(trace, row=1, col=1)
for trace in fig2.data:
    fig.add_trace(trace, row=1, col=2)
for trace in fig3.data:
    fig.add_trace(trace, row=1, col=3)
for trace in fig4.data:
    fig.add_trace(trace, row=2, col=1)
for trace in fig5.data:
    fig.add_trace(trace, row=2, col=2)
for trace in fig6.data:
    fig.add_trace(trace, row=2, col=3)
for trace in fig7.data:
    fig.add_trace(trace, row=3, col=1)
for trace in fig8.data:
    fig.add_trace(trace, row=3, col=2)



# Actualizar el layout
fig.update_layout(title_text='Subplots with Plotly Express')

# Mostrar la figura
fig.show()